In [14]:
!pip install --upgrade ctab-xtra-dp==3.5.0

In [15]:
from ctab_xtra_dp import CTAB_XTRA_DP , load_demo, evaluation
import numpy as np
import pandas as pd


In [ ]:
df = pd.read_csv('../datasets/fraud.csv')

In [17]:
missing_values_columns = ["prev_address_months_count", "current_address_months_count", "device_distinct_emails_8w"]
df[missing_values_columns] = df[missing_values_columns].replace(-1, np.nan)

In [18]:
df.isna().sum()

fraud_bool                            0
income                                0
name_email_similarity                 0
prev_address_months_count           701
current_address_months_count          2
customer_age                          0
payment_type                          0
date_of_birth_distinct_emails_4w      0
employment_status                     0
credit_risk_score                     0
email_is_free                         0
housing_status                        0
phone_home_valid                      0
has_other_cards                       0
proposed_credit_limit                 0
device_os                             0
keep_alive_session                    0
device_distinct_emails_8w             0
dtype: int64

In [19]:
train, test = evaluation.split_data(df, 0.2,problem="regression",random_state=22)

In [20]:
categorical_columns =  ["fraud_bool","payment_type","employment_status","email_is_free","housing_status","device_os","phone_home_valid","has_other_cards","keep_alive_session"]
mixed_columns = {}
general_columns = ['date_of_birth_distinct_emails_4w','credit_risk_score']

In [ ]:
def fit_and_save_constraints(df, epsilon_budget_list,categorical_columns, mixed_columns, general_columns,epochs = 100):
    filepath = "synthetic_data/"
    filename = "fraud"
    synthetic_data_list = []
    for epsilon in epsilon_budget_list:
        
        synthesizer = CTAB_XTRA_DP(df,
                          categorical_columns=categorical_columns, 
                          mixed_columns=mixed_columns, 
                          general_columns=general_columns,
                          problem_type= {"Classification": "fraud_bool"},
                          dp_constraints= {"epsilon_budget": epsilon}
        )
        synthesizer.fit(epochs)
        synthetic_data = synthesizer.generate_samples(df.shape[0])
        epsilon_text = epsilon if epsilon is not None else "None"
        synthetic_data.to_csv(f'{filepath}{filename}_epochs_{epochs}_epsilon_{epsilon_text}.csv', index=False)





fit_and_save_constraints(train, [None,15,5,1], categorical_columns, mixed_columns, general_columns, epochs = 1)




    

100%|██████████| 1/1 [00:03<00:00,  3.77s/it]


Finished training in 5.188199043273926  seconds.
No delta provided, setting delta: 0.000125
